In [91]:
import fitsio
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from desitarget.sv1 import sv1_targetmask

#1. BGS MAIN: 
#(RMAG < 19.5) 
#This is the same as the previous nominal BGS main/bright

#2. BGS FAINT: 
#66% of targets with (19.5 < R_MAG < 20.0) & (RFIBER_COLOR_CUT)
#where
#RFIBER_COLOR_CUT = (RFIBER < 20.75) or ( (RFIBER < 21.5) & (COLOR > 0) )
#COLOR = (Z_MAG - W1_MAG) - 3/2.5 * (G_MAG - R_MAG) + 1.2
#These targets are lower priority than 1. BGS MAIN

#3. BGS FAINT HIP: 
#33% of targets with the same selection as above with the same
#high priority as 1. BGS MAIN. 

In [92]:
data = Table.read("targets/mtl_bright_sv1_onepct.fits")
print(len(data))

1198252


In [95]:
def BGS(indata, datafile='test.fits'):
    data = indata.copy()
    flux_r = data["FLUX_R"]/data["MW_TRANSMISSION_R"]
    flux_g = data["FLUX_G"]/data["MW_TRANSMISSION_G"]
    flux_z = data["FLUX_Z"]/data["MW_TRANSMISSION_Z"]
    flux_W1 = data["FLUX_W1"]/data["MW_TRANSMISSION_W1"]
    flux_W2 = data["FLUX_W2"]/data["MW_TRANSMISSION_W2"]
    fiber_r = data["FIBERFLUX_R"]/data["MW_TRANSMISSION_R"]

    gmag = 22.5 - 2.5 * np.log10(flux_g.clip(1e-7))
    rmag = 22.5 - 2.5 * np.log10(flux_r.clip(1e-7))
    zmag = 22.5 - 2.5 * np.log10(flux_z.clip(1e-7))
    w1mag = 22.5 - 2.5 * np.log10(flux_W1.clip(1e-7))
    w2mag = 22.5 - 2.5 * np.log10(flux_W2.clip(1e-7))
    rfibermag = 22.5 - 2.5 * np.log10(fiber_r.clip(1e-7))
    
    rr = np.random.random(len(data))
    
    is_bgs_main  = rmag < 19.5
    
    color = (zmag - w1mag) - 3.0/2.5 * (gmag - rmag) + 1.2
    rfiber_color_cut = (rfibermag < 20.75) | ((rfibermag < 21.5) & (color>0))
    
    is_bgs_faint = (rmag>19.5) & (rmag<20.0) & (rfiber_color_cut) & (rr < 0.66)

    is_bgs_hip = (rmag>19.5) & (rmag<20.0) & (rfiber_color_cut) & (~(rr < 0.66))
    
    is_bgs = is_bgs_main | is_bgs_faint | is_bgs_hip
    
    print(np.count_nonzero(is_bgs_main), np.count_nonzero(is_bgs_faint), np.count_nonzero(is_bgs_hip))
    was_bgs = (data['SV1_DESI_TARGET']&sv1_targetmask.desi_mask['BGS_ANY'])!=0
    data['PRIORITY_INIT'][was_bgs]=0
    data['PRIORITY'][was_bgs]=0
    data['NUMOBS_MORE'][was_bgs]=0
    data['NUMOBS_INIT'][was_bgs]=0
    
    data['PRIORITY_INIT'][is_bgs_faint] = 2000
    data['PRIORITY'][is_bgs_faint] = 2000
    data['PRIORITY_INIT'][is_bgs_hip] = 2100
    data['PRIORITY'][is_bgs_hip] = 2100
    data['PRIORITY_INIT'][is_bgs_main] = 2100
    data['PRIORITY'][is_bgs_main] = 2100
    data['NUMOBS_MORE'][is_bgs] = 1
    data['NUMOBS_INIT'][is_bgs] = 1

    ii = (data['SV1_DESI_TARGET']&sv1_targetmask.desi_mask['BGS_ANY'])!=0
    print(np.count_nonzero(ii), np.count_nonzero(is_bgs))
    
    data['SV1_BGS_TARGET'][~is_bgs] = 0
    data['SV1_BGS_TARGET'][is_bgs_faint] = sv1_targetmask.bgs_mask['BGS_FAINT']
    data['SV1_BGS_TARGET'][is_bgs_hip] = sv1_targetmask.bgs_mask['BGS_LOWQ']
    data['SV1_BGS_TARGET'][is_bgs_main] = sv1_targetmask.bgs_mask['BGS_BRIGHT']
    
    data = data[~(~is_bgs & was_bgs)] # exclude everything that was bgs before but it's not BGS anymore
    print(len(data))
    print('writing to ', datafile)
    data.write(datafile, overwrite=True)

In [96]:
BGS(data, datafile='targets/mtl_bright_newbgs_sv1_onepct.fits')

587374 128429 66326
873360 782129
782236
writing to  targets/mtl_bright_newbgs_sv1_onepct.fits
